# Importamos librerias - Creamos el dataframe de pericias

In [260]:
import pandas as pd
import numpy as np
from pprint import pprint

df = pd.read_csv('pericias_medicas.csv', sep=',',  encoding='utf-8')
#convierte en string los datos de la columna text
df = df['text'].apply(str)
df1 = df.head(20)

## Limpieza de texto

In [264]:
# Limpieza de texto
# Primera etapa de la limpieza de texto
import re, string, unicodedata
'''
Se hace la eliminacion de texto entre corchetes, acentos, signos de puntuacion (excepto . y :), palabras con numeros.
Se eliminan los espacios de sobra
Se eliminan \r, \t, \v, \f, \a
'''
def limpiarTexto1(txt: str, bert=False, nums=False) -> str:
    """
    Elimina caracteres no deseados
    Params:
        **txt**:texto a ser limpiado de caracteres no desaeados
    """
    if nums:
        txt = re.sub(r'\d+', ' ', txt)
    if not bert:
        txt = txt.translate(str.maketrans(
            'áéíóúýàèìòùÁÉÍÓÚÀÈÌÒÙÝ', 'aeiouyaeiouAEIOUAEIOUY'))

    '''
    Eliminamos caracteres especiales: tabulador orizontal(\t), tabulador vertical(\v), 
    retorno de carro(\r), avance de pagina(\f), 
    caracter de retroceso: Marca el límite de una palabra(\b), 
    '''
    txt = txt.replace('\r', ' ').replace("\v", ' ').replace(
        "\t", ' ').replace("\f", ' ').replace("\a", ' ').replace("\b", ' ')
    txt = re.sub(' +', ' ', txt)
    txt = txt.strip()
    return txt
df1.apply(str)
df1 = df1.apply(limpiarTexto1)


In [265]:
# Segunda fase de limpieza
# Se eliminan todos los elementos que meten ruido al texto y que no fueron eliminados en la fase de limpieza 1.
import re

def limpiarTexto2(text):
    text = re.sub('^ ',' ',text)
    text = re.sub('\n +\n','\n',text)
    text = re.sub(' +\n\n','\n',text)
    text = re.sub('\n\n+','\n',text)
    text = re.sub(' \n','\n',text)
    text = re.sub('\d\n','',text)
    text = re.sub('\x0c','',text)
    text = re.sub('\u200b\n','',text)
    text = re.sub('\d{3,100}','',text)
    text = re.sub('\d+-\d+','',text)
    text = re.sub('\x0c','',text)
    text = re.sub('[nN]º|[nN][. ]º','',text)
    text = re.sub('[º<>/]','',text)
    text = re.sub('\d{3,100}','',text)
    text = re.sub('[a-zA-z-.]+@[a-zA-Z]+.com','',text)
    return text

dfLimpio = df1.apply(limpiarTexto2)

In [266]:
# Tercera fase de limpieza
# Eliminamos las lineas que no son de utilidad para el analisis o que van a afectar los resultados del mismo.
# Ejemplo de linea eliminada: las lineas que comienzan con "Se encuentra contestada en....."
import re

def limpiarTexto3(text):
    text = re.sub('[a-z1-9.]+[).-] [s|S]e encuentra contestad[a|o] .+[. \n]','',text)
    text = re.sub('[0-9]+[. ]+[yY]a fue contestado.+[.\n]','',text)
    text = re.sub('[fF]oja [1-9].+\n', '', text)
    text = re.sub('[pP]regunta[ 0-9]+[)].+\n|[rR]espuesta[ 0-9]+[)].+\n','',text)#elimina oraciones comenzadas en preguta/respuesta.
    return text

dfLimpio = dfLimpio.apply(limpiarTexto3)
dfLimpio = dfLimpio.apply(limpiarTexto2)

# Busqueda de titulos

In [268]:
# Busqueda de titulos
# Se crea un nuevo dataframe de titulos según distintos criterios

# busca titulos que comienzan con numeros romanos
def buscarTitulosRomanos(text):
    # Expresiones regulares para encontrar los numeros romanos
    tituloNroRomano =re.compile(r'(I{1,3}|IV|V|VI{1,3}|IX|X)[-.)][\w\s]+[:\n]')
    titulosRomanosEncontrados = []

    for m in tituloNroRomano.finditer(text):
        titulosRomanosEncontrados.append(m.group())

    return titulosRomanosEncontrados

# titulosConNroRomano: lista que guarda los titulos que comienzan con numeros romanos   
titulosConNroRomano=[]
for expediente in dfLimpio:
    titulosConNroRomano.append(buscarTitulosRomanos(expediente))

#pprint(titulosConNroRomano[9])

In [274]:
# Busca titulos que comienzan con numeros latinos
def buscarTitulosLatinos(text):
    # Expresion regular para encontrar titulos que comienzan con numeros
    tituloNroLatino =re.compile(r'[1-9][-.)][\w\s]+[:\n]')
    titulosLatinosEncontrados = []

    for m in tituloNroLatino.finditer(text):
        titulosLatinosEncontrados.append(m.group())

    return titulosLatinosEncontrados

# titulosConNroLatino: lista que guarda los titulos que comienzan con numeros latinos
titulosConNroLatino=[]
for expediente in dfLimpio:
    titulosConNroLatino.append(buscarTitulosLatinos(expediente))
#pprint(titulosConNroLatino[1])

In [ ]:
# Busca titulos en mayusculas
def buscarTitulosMayusculas(text):
    # Expresion regular para encontrar titulos en mayusculas.
    tituloMayusculas =re.compile(r'([1-9]|I{1,3}|IV|V|VI{1,3}|IX|X)[-.)][A-Z -]+[:.\n]')
    titulosMayusculasEncontrados = []

    for m in tituloMayusculas.finditer(text):
        titulosMayusculasEncontrados.append(m.group())

    return titulosMayusculasEncontrados
# titulosMAyuscula: lista que guarda los titulos en mayusculas
titulosMayusculas=[]
for expediente in dfLimpio:
    titulosMayusculas.append(buscarTitulosMayusculas(expediente))


#### Eliminacion de stopwords y lematizacion.

In [11]:
import nltk
import re

stops = nltk.corpus.stopwords.words('spanish')

#elimino stopwords
def remove_stops(texto: str) -> str:
    """
    Función que elimina stopwords
    Params:
        **texto**:texto a ser limpiado de stopwords

    """
    texto = [
        i for i in texto.split() if i not in stops
    ]
    return texto

df1 = df1.apply(remove_stops)

#print(df1)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 

documento = df1.apply(str)  
# crear la transformación  
vectorizer = CountVectorizer() 
# tokenizar y construir el vocabulario
vectorizer.fit(documento) 
  
print("Vocabulary: ", vectorizer.vocabulary_) 
# codificador de documentos
vector = vectorizer.transform(documento) 

# resumir vector codificado
#print("El documento codificado es:") 
#print(vector.shape)
#print(type(vector))
#print(vector.toarray())

In [ ]:
# r'.\n +'[0-9]+[-. ]+[a-zA-z ]+'

#.\n +'([vV][.-]+ [A-Za-z ]+)|([iI{0,3}][vV]+[.-]+ [A-Za-z ]+)